In [1]:
# metabolic model open
import cobra

s12 = cobra.io.read_sbml_model('~\\input\\iSH1474.xml')
kt2440 = cobra.io.read_sbml_model('C:\\Users\\twall\\Downloads\\iJN1463.xml')
s12.objective = 'BIOMASS_S12_WT'
kt2440.objective = 'BIOMASS_KT2440_WT3'

In [2]:
r_id = ['GND','PGI','EDA','TKT2','FBA','PGK','ENO','PYK','PDH','PPCK','ME2','MDH','FUM','PC','ICDHyr','ASPTA']
exp = [0.6, -0.4, 7, 0.1, -1.1, -4.7, 3.7, 4.4, 6.9, 1.1, 2.6, 5.1, 4.8, 5.4, 5.6]

for r in r_id:
    print(s12.reactions.get_by_id(r))

GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c
PGI: g6p_c <=> f6p_c
EDA: 2ddg6p_c --> g3p_c + pyr_c
TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
FBA: fdp_c <=> dhap_c + g3p_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c
ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
FUM: fum_c + h2o_c <=> mal__L_c
PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c


In [3]:
s12.reactions.get_by_id('EX_glc__D_e').lower_bound = -6.5
kt2440.reactions.get_by_id('EX_glc__D_e').lower_bound = -6.5

In [4]:
import pandas as pd
import numpy as np

s12_c13 = []
kt2440_c13 = []

for r in r_id:
    if r == 'ICDHyr':
        ICDHyr=kt2440.optimize().fluxes[r]
    elif r == 'ASPTA':
        ASPTA=kt2440.optimize().fluxes[r]
    else:
        kt2440_c13.append(kt2440.optimize().fluxes[r])
kt2440_c13.append(ICDHyr-ASPTA)
kt2440_c13.append(np.corrcoef(kt2440_c13,exp)[0,1])

for r in r_id:
    if r == 'ICDHyr':
        ICDHyr=s12.optimize().fluxes[r]
    elif r == 'ASPTA':
        ASPTA=s12.optimize().fluxes[r]
    else:
        s12_c13.append(s12.optimize().fluxes[r])
s12_c13.append(ICDHyr-ASPTA)
s12_c13.append(np.corrcoef(s12_c13,exp)[0,1])

index=['GND','PGI','EDA','TKT2','FBA','PGK','ENO','PYK','PDH','PPCK','ME2','MDH','FUM','PC','ICDHyrASPTA','r']

df = pd.DataFrame({'s12_c13':s12_c13,'kt2440_c13':kt2440_c13},index=index)
df

,s12_c13,kt2440_c13
GND,0.891788,0.876962
PGI,-0.022558,-0.022099
EDA,5.608212,5.623038
TKT2,-0.092853,-0.092745
FBA,0.000000,0.000000
PGK,-5.470036,-5.486338
ENO,4.511763,4.302286
PYK,3.964083,3.489437
PDH,7.621426,0.000000
PPCK,0.000000,0.000000
